# Traveling Salesman Problem

José Luis Lobera del Castillo <br>
Rafael Andrade Ruíz Capetillo

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import random
import copy

In [4]:
def read_csv_file(path):
    df = pd.read_csv(path, index_col=0)
    node_names = df.columns
    distance_matrix = df.values
    node_position_dict = dict(zip(node_names, list(range(len(node_names)))))

    return node_names, distance_matrix, node_position_dict


In [5]:
def GenerateNodes(n,lim):
    array_x= np.random.randint(lim, size=n)
    array_y= np.random.randint(lim, size=n)
    abecedario=['A','B','C','D','E','F','G','H','I','J','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    array_names=[]

    for i in range(0,n):
        array_names.append(abecedario[i])

    #Para distancia exacta quitar que sea int 
    dist_matrix=np.zeros((n,n), dtype=int)
    for i in range(0,n):
        for j in range(0,n):
            if i!=j:
                dist_matrix[i,j]=math.sqrt((array_y[i]-array_y[j])**2+(array_x[i]-array_x[j])**2)
                
    #print(dist_matrix)
    dNodes={}
    for i in range(0,n):
        dNodes[array_names[i]]=i
    #print(dNodes)
    
    return array_names,dNodes,array_x,array_y,dist_matrix

In [6]:
# Use only with generated nodes

def PrintNodes(names,x,y):
    fig=plt.figure()
    ax=fig.add_axes([0,0,1,1])
    ax.scatter(x, y, color='r')

    for i in range(0,len(names)):
        ax.annotate(names[i], (x[i], y[i]))
        
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('NODES')
    
    plt.show()

In [7]:
def objectiveFunction(node_names, distance_matrix, individual, node_position_dict):
    individual = np.array(individual)

    if(individual[0] != individual[-1]): return -1                  # First and Last must be equal
    if(len(individual) != len(node_names)+1): return -1             # Shorter individual than expected
    if(len(node_names) != len(np.unique(individual))): return -1    # Unique values only
    #if(sorted(individual[:-1]) != sorted(node_names)): return -1

    distance = 0
    for i, node in enumerate(individual):
        if(i+1 >= len(individual)): break
    
        distance += distance_matrix[ node_position_dict[ individual[ i ]], node_position_dict[ individual[ i+1 ] ]]
        #print(i, node_position_dict[ individual[ i ]])
        #print(i, node_position_dict[ individual[ i+1 ]])
    return distance

In [8]:
node_names, distance_matrix, node_position_dict = read_csv_file('adjacency_matrix.csv')

## Genetic Algorithm

### Aux Funcions:

In [9]:
# Creates a permutation of all nodes
# Returns: List of the path,total distance

def GetRandomChromosome():
    individual = np.random.permutation(node_names[1:])
    individual = np.append(individual, node_names[0])
    individual = np.append(np.array([node_names[0]]), individual)
    distance = objectiveFunction(node_names, distance_matrix, individual, node_position_dict)
    return [individual, distance]

In [10]:
# Creates a population of random chromosomes
# Returns: List of Chromosomes

def GetPopulation(n = 0):
    return [GetRandomChromosome() for _ in range(n)] # O(n)

#### Roulette Selection

In [11]:
# Sums the total weights of chromosomes
# Returns a list of proportions

def GetProportion(population):
    total = 0
    for chromosome in population:
        total += 1/chromosome[1]
    
    proportion = []
    for chromosome in population:
        proportion.append((1/chromosome[1])/total)

    return proportion

In [12]:
def RouletteSelection(proportions):
    selection = random.random()

    acum = 0

    for i, prop  in enumerate(proportions):
        acum += prop
        if selection < acum:
            return i


##### Roulette Selected Chromosome

In [13]:
population = GetPopulation(10)

proportion = GetProportion(population)

# The way of getting a Roulette Selected Chromosome
selected = population[RouletteSelection(proportion)]
print(selected)

[array(['A', 'H', 'E', 'I', 'G', 'D', 'B', 'J', 'F', 'C', 'A'],
      dtype=object), 459]


#### Tournament Selection

In [14]:
def TournamentSelection(population, k=2):
    community = random.sample(population, k)
    localElite = population[0]
    
    for chromosome in community:
        if chromosome[1] < localElite[1]:
            localElite = chromosome
            
    return localElite

##### Tournament Selected Chromosome

In [15]:
population = GetPopulation(10)

TournamentSelection(population, 5)

[array(['A', 'B', 'G', 'D', 'H', 'J', 'F', 'I', 'E', 'C', 'A'],
       dtype=object),
 425]

In [16]:
def pointCrossover(A, B, rp, n, node_names, distance_matrix, node_position_dict):
    aux = random.random()
    if aux <= rp:
        crosspoint = random.randint(0,n)
        offspring = copy.deepcopy(A[0][:crosspoint])
        offspring = np.append(offspring,copy.deepcopy(B[0][crosspoint:]))
        dist = objectiveFunction(node_names, distance_matrix, offspring, node_position_dict)
        
        if dist == -1: return copy.deepcopy(A)
        return [offspring,dist]
        
    else:
        return copy.deepcopy(A)
        
    

In [17]:
def swapMutation(A, rp, n, node_names, distance_matrix, node_position_dict):
    elementA = random.randint(1, n-1)
    elementB = random.randint(1, n-1)
    aux = A[0][elementA]
    A[0][elementA] =A [0][elementB]
    A[0][elementB] = aux
    A[1] =objectiveFunction(node_names, distance_matrix, A[0], node_position_dict)
    

In [18]:
def getElite(A, B):
    if A is None: return B
    if B is None: return A

    distA = objectiveFunction(node_names, distance_matrix, A[0], node_position_dict)
    distB = objectiveFunction(node_names, distance_matrix, B[0], node_position_dict)
    if distA < distB:
        print('New Best')
        return A
    else:
        return B

### Main Algorithm

In [19]:
def geneticAlgorithm(nCities, N, G, Pr, Pm):
    eliteIndividual = None

    fatherPopulation = GetPopulation(N) # First Gen
    sonPopulation = []

    actualGeneration = 0

    while actualGeneration < G: # OR we haven't found a good solution
        
        for _ in range(100):
            G11 = TournamentSelection(fatherPopulation)
            G12 = TournamentSelection(fatherPopulation)

            G21 = pointCrossover(G11, G12, Pr, nCities, node_names, distance_matrix, node_position_dict)
            
            swapMutation(G21, Pr, nCities, node_names, distance_matrix, node_position_dict)

            sonPopulation.append(G21)
            
            eliteIndividual = getElite(G21, eliteIndividual)

        for x in sonPopulation:
            if x[1] == -1: print("-1")
        
        fatherPopulation = sonPopulation
        sonPopulation = []

        actualGeneration += 1
    
    return eliteIndividual


In [26]:
nCities = 10
N = 30
G = 100
Pr = 0.8
Pm = 0.3

print(geneticAlgorithm(nCities, N, G, Pr, Pm))

New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
New Best
[array(['A', 'H', 'B', 'I', 'F', 'J', 'D', 'G', 'C', 'E', 'A'],
      dtype=object), 272]
